In [ ]:
from google.colab import drive
import os

drive.mount('/content/gdrive')
os.environ['dir'] = "/content/gdrive/My Drive/AptosData/aptos2019-blindness-detection"
%cd /content/gdrive/My Drive/AptosData/aptos2019-blindness-detection

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/1VoIiqaGjfOUECGveuZ9M_wlPyn5DCNZd/AptosData/aptos2019-blindness-detection


In [ ]:
#Pip install libraries needed
!pip install -U efficientnet
!pip install q tensorflow
!pip install q keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
import os
import glob
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm


from keras.layers import GlobalAveragePooling2D
from keras import Sequential
import efficientnet.keras as efn
import numpy as np
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Set paths
root_dir = 'filtered_images'  # This is inside your current working directory
output_dir = 'split_dataset'

# Map folder names to class labels
class_map = {
    'No_DR': 0,
    'Mild': 1,
    'Moderate': 2,
    'Severe': 3,
    'Proliferate_DR': 4
}

# Gather image paths and labels
image_paths = []
labels = []

for folder_name, class_id in class_map.items():
    folder_path = os.path.join(root_dir, folder_name)
    image_list = glob.glob(os.path.join(folder_path, '*.png'))  # or .png if needed
    image_paths.extend(image_list)
    labels.extend([class_id] * len(image_list))

df = pd.DataFrame({'filepath': image_paths, 'label': labels})

In [ ]:
# Perform stratified train/val/test split (70/15/15)
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

# Copy files to destination folders
def copy_images(subset_df, subset_name):
    for _, row in tqdm(subset_df.iterrows(), total=len(subset_df), desc=f'Copying {subset_name}'):
        label = row['label']
        label_name = [k for k, v in class_map.items() if v == label][0]
        dest_path = os.path.join(output_dir, subset_name, label_name)
        os.makedirs(dest_path, exist_ok=True)
        shutil.copy(row['filepath'], os.path.join(dest_path, os.path.basename(row['filepath'])))

# Start copying
copy_images(train_df, 'train')
copy_images(val_df, 'val')
copy_images(test_df, 'test')

Copying train:   0%|          | 0/2563 [00:00<?, ?it/s]

Copying val:   0%|          | 0/549 [00:00<?, ?it/s]

Copying test:   0%|          | 0/550 [00:00<?, ?it/s]

In [ ]:
summary = {
    'train': train_df['label'].value_counts().sort_index(),
    'val': val_df['label'].value_counts().sort_index(),
    'test': test_df['label'].value_counts().sort_index()
}
summary_df = pd.DataFrame(summary).rename(index={v: k for k, v in class_map.items()})
summary_df
